# Install openvino libs

In [1]:
!pip install /kaggle/input/runtimes-onnx-openvino/openvino/openvino_telemetry-2025.1.0-py3-none-any.whl --no-index --find-links /kaggle/input/runtimes-onnx-openvino/openvino
!pip install /kaggle/input/runtimes-onnx-openvino/openvino/openvino-2025.0.0-17942-cp310-cp310-manylinux2014_x86_64.whl --no-index --find-links /kaggle/input/runtimes-onnx-openvino/openvino

Looking in links: /kaggle/input/runtimes-onnx-openvino/openvino
Processing /kaggle/input/runtimes-onnx-openvino/openvino/openvino_telemetry-2025.1.0-py3-none-any.whl
Looking in links: /kaggle/input/runtimes-onnx-openvino/openvino
Processing /kaggle/input/runtimes-onnx-openvino/openvino/openvino-2025.0.0-17942-cp310-cp310-manylinux2014_x86_64.whl


# Imports

In [2]:
import os
import tqdm
import ast
from glob import glob
import math
import joblib

from scipy.ndimage import convolve1d
import pandas as pd
import numpy as np
import librosa
import timm
import torch
import torch.nn as nn
from torch.nn import functional as F
import torchaudio.transforms as T
from openvino.runtime import Core
import openvino as ov

/usr/lib/python3.10/importlib/util.py:247: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  self.__spec__.loader.exec_module(self)


# SED Model Architecture

In [3]:
def gem_freq(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), 1)).pow(1.0 / p)


class GeMFreq(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return gem_freq(x, p=self.p, eps=self.eps)


class AttHead(nn.Module):
    
    def __init__(self, in_chans, p=0.5, num_class=397, hidden_dim=512):
        super().__init__()
        
        self.pooling = GeMFreq()
        self.dense_layers = nn.Sequential(
                nn.Dropout(p / 2),
                nn.Linear(in_chans, hidden_dim),
                nn.ReLU(),
                nn.Dropout(p),
            )
        self.attention = nn.Conv1d(
            in_channels=hidden_dim,
            out_channels=num_class,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.fix_scale = nn.Conv1d(
            in_channels=hidden_dim,
            out_channels=num_class,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )

    def forward(self, feat):

        feat = self.pooling(feat).squeeze(-2).permute(0, 2, 1)  # (bs, time, ch) 

        feat = self.dense_layers(feat).permute(0, 2, 1)  # (bs, 512, time)
        
        # time_att_logit = torch.tanh(self.attention(feat))

        framewise_logit = self.fix_scale(feat)
        framewise_prob = torch.sigmoid(framewise_logit)
        
        # clipwise_prob = torch.sum(
        #     segmentwise_prob * torch.softmax(time_att_logit, dim=-1),
        #     dim=-1,
        # )
        output = {
            # "clipwise_prob": clipwise_prob,
            # "time_att_logit": time_att_logit,
            "framewise_logit": framewise_logit
        }
        return output
        
        
class NormalizeMelSpec(nn.Module):
    
    def __init__(self, norm_type="default", eps=1e-6, constant=80):
        super().__init__()
        self.eps = eps
        self.norm_type = norm_type
        self.constant = constant

    def forward(self, X):
        
        if self.norm_type == "default":
            mean = X.mean((1, 2), keepdim=True)
            std = X.std((1, 2), keepdim=True)
            
            Xstd = (X - mean) / (std + self.eps)
            norm_max = torch.amax(Xstd, dim=(1, 2), keepdim=True)
            norm_min = torch.amin(Xstd, dim=(1, 2), keepdim=True)
            return (Xstd - norm_min) / (norm_max - norm_min + self.eps)
            
        elif self.norm_type == "top_db":
            X = (X + 80) / 80
            return X

        elif self.norm_type == "constant":
            X = X / self.constant
            return X
            
        
class SpecFeatureExtractor(nn.Module):
    
    def __init__(
        self,
        n_fft: int,
        hop_length: int,
        win_length = None,
        sample_rate=200,
        f_max=20,
        f_min=0.5,
        n_mels=128,
        top_db=120,
        normalized=False,
        sample_mel_normalize=None,
        output_size=(256, 256),
    ):
        
        super().__init__()

        self.feature_extractor = nn.Sequential()
        self.feature_extractor.append(
                    T.MelSpectrogram(sample_rate=sample_rate, 
                                     normalized=normalized,
                                     n_fft=n_fft, 
                                     hop_length=hop_length, 
                                     win_length=win_length, 
                                     f_max=f_max,
                                     n_mels=n_mels, 
                                     f_min=f_min)
        )
        
        self.feature_extractor.append(T.AmplitudeToDB(top_db=top_db))
        
        if sample_mel_normalize is not None:
            self.feature_extractor.append(NormalizeMelSpec(norm_type=sample_mel_normalize))

        if output_size is not None:
            self.resize = nn.UpsamplingBilinear2d(size=output_size)
        else:
            self.resize = None

    def norm(self, x: torch.Tensor):
        img_norm = self.feature_extractor[-1](x)
        return img_norm
            
    def forward(self, x: torch.Tensor, without_norm=False) -> torch.Tensor:
        if without_norm:
            img = self.feature_extractor[:-1](x)
        else:
            img = self.feature_extractor(x)
        if self.resize is not None:
            img = self.resize(img.unsqueeze(1)).squeeze(1)

        return img
    

class CLEFClassifierSED(nn.Module):
    
    def __init__(self, 
                 config, 
                 disable_spectr_generator=False):
        
        super(CLEFClassifierSED, self).__init__()
        
        spectrogram_config = config['spectrogram']
        backbone_config = config['backbone']
        head_config = config['head']
        
        self.mel_spectr_generator = SpecFeatureExtractor(**spectrogram_config)
                
        self.backbone = timm.create_model(backbone_config['backbone_name'],
                                          pretrained=backbone_config['pretrained'],
                                          features_only=True)
        backbone_dim = self.backbone.feature_info.channels()[-1]
        
        self.head = AttHead(in_chans=backbone_dim, 
                            p=head_config['dropout'], 
                            num_class=head_config['num_classes'])
        self.infer_duration = head_config['infer_duration']
        self.duration = head_config['duration']
        self.inference_type = config['inference_type']

        self.multilabel_to_train_labels = config.get('multilabel_to_train_labels', "one2one")

    def get_head_preds(self, input, tta_delta=2, sigmoid=True):
        
        head_output = self.head(input)
        feat_time = input.shape[-1]
        framewise_prob = head_output["framewise_logit"].sigmoid().numpy()
        num_segments, num_labels, frames_per_segment = framewise_prob.shape
        
        step = int(feat_time * (self.infer_duration / self.duration))
        
        pad_frames = (frames_per_segment - step) // 2
        if (frames_per_segment - step) % 2 != 0:
            extra_pad = 1
        else:
            extra_pad = 0
        framewise_ss_len = frames_per_segment + step*(num_segments - 1)
        
        
        framewise_ss_preds = np.zeros((framewise_ss_len, num_segments, num_labels), dtype=np.float16)
        framewise_ss_preds_mask = np.zeros((framewise_ss_len, 1), dtype=np.float16)
        
        for segment_ind in range(num_segments):
            framewise_ss_preds[segment_ind*step: (segment_ind*step) + frames_per_segment, segment_ind] += framewise_prob[segment_ind].T
            framewise_ss_preds_mask[segment_ind*step: (segment_ind*step) + frames_per_segment] += 1
        
        if self.inference_type in ["overlap_average_max", "overlap_average_max_delta"]:
            framewise_ss_preds = framewise_ss_preds.sum(1)
            framewise_ss_preds = framewise_ss_preds / framewise_ss_preds_mask
            
        elif self.inference_type == "overlap_max":
            framewise_ss_preds = framewise_ss_preds.max(1)

        framewise_ss_preds = framewise_ss_preds[pad_frames:-pad_frames - extra_pad]
        segmentwise_preds = framewise_ss_preds.reshape(num_segments, step, num_labels).max(1)
        if self.inference_type == "overlap_average_max_delta": 
            segmentwise_preds *= 0.5
            for segment_ind in range(num_segments):
                if segment_ind == 0:
                    segmentwise_preds[segment_ind] += framewise_ss_preds[segment_ind*step: (segment_ind+1)*step].max(0) * 0.25
                else:
                    segmentwise_preds[segment_ind] += framewise_ss_preds[segment_ind*step - tta_delta: (segment_ind+1)*step - tta_delta].max(0) * 0.25
    
                if segment_ind == (num_segments -1):
                    segmentwise_preds[segment_ind] += framewise_ss_preds[segment_ind*step: (segment_ind+1)*step].max(0) * 0.25
                else:
                    segmentwise_preds[segment_ind] += framewise_ss_preds[segment_ind*step + tta_delta: (segment_ind+1)*step + tta_delta].max(0) * 0.25
        
        return segmentwise_preds
            

    def forward(self, 
                input, 
                labels=None,
                train=False):
    
        if not train:
            return {"preds": self.inference(input)}
                
        x = self.mel_spectr_generator(input)    
        
        if self.augmentations is not None and train:
            x = self.augmentations(x)
    
        x = torch.stack([x, x, x], 1)
        
        # if train and labels is not None:
        #     x, labels = horizontal_cutmix(x, labels)    
        
        x = self.backbone(x)[-1]
        head_output = self.head(x)
        if train and labels is not None:
            loss = 0.5 * self.critarion(torch.logit(head_output["clipwise_prob"]), labels) + 0.5 * self.critarion(head_output["segmentwise_logit"].max(2)[0], labels)
        else:
            loss = None
        output = {
            "preds": torch.logit(head_output["clipwise_prob"]),
            "loss": loss,
            "labels": labels
        }
        return output

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Inference Dataset, Inference Engine

In [4]:
class InferenceDataset:
    def __init__(self, all_waves, feature_extractor, full_signal=True, hop_length=1252, img_size=(224, 512), sample_rate=32000, duration_sec=5, slice_step_sec=5, num_segments_sample=12, res_type="kaiser_best", separate_norm=False):
        
        self.all_waves = all_waves
        self.all_filenames = list(all_waves.keys())
        
        self.feature_extractor = feature_extractor
        self.sample_rate = sample_rate
        self.duration_timestamps = sample_rate*duration_sec
        self.slice_step_timestamps = sample_rate*slice_step_sec
        self.num_segments_sample = num_segments_sample
        self.res_type = res_type
        
        self.full_signal = full_signal
        self.hop_length = hop_length
        self.img_size = img_size
        self.separate_norm = separate_norm

    def __len__(self):
        return len(self.all_filenames)
        
    def split_wave(self, wave):
        if self.slice_step_timestamps != self.duration_timestamps:
            pad_side_length = int((self.duration_timestamps - self.slice_step_timestamps) / 2)
            wave = np.pad(wave, (pad_side_length, pad_side_length))
            segments = [librosa.util.normalize(wave[i*self.slice_step_timestamps: i*self.slice_step_timestamps + self.duration_timestamps]) for i in range(self.num_segments_sample)]
        else:
            segments = [librosa.util.normalize(wave[i*self.duration_timestamps: (i+1)*self.duration_timestamps]) for i in range(self.num_segments_sample)] 
        segments = np.stack(segments, 0)
        return segments

    def prepare_mel_specs_from_segments(self, wave):
        segments = self.split_wave(wave=wave)
        mel_specs = self.feature_extractor(torch.tensor(segments))
        return mel_specs

    def prepare_mel_specs_from_full_wave(self, wave):
        
        def get_wave_len(hop_length, time_bins):
                return (time_bins -1) * hop_length
            
        total_time_bins = self.img_size[1] + (self.num_segments_sample - 1)*(self.img_size[1]*self.slice_step_timestamps/self.duration_timestamps)
        
        if not self.separate_norm:
            total_wave_len = get_wave_len(hop_length=self.hop_length, time_bins=total_time_bins)
            pad_side_length = math.ceil(((total_wave_len - len(wave)) / 2))
            wave = np.pad(wave, (pad_side_length, pad_side_length))
            
        mel_spec_full = self.feature_extractor(torch.tensor(wave.reshape(1, -1)), without_norm=self.separate_norm)
        
        if self.separate_norm:
            pad_side_length = int((total_time_bins- mel_spec_full.shape[-1]) // 2)
            extra_pad = 0
            if (total_time_bins - mel_spec_full.shape[-1]) % 2 !=0:
                extra_pad = 1
            mel_spec_full = F.pad(mel_spec_full, pad=(pad_side_length, pad_side_length + extra_pad, 0, 0), mode='constant', value=mel_spec_full.min())

        slice_step = int((self.img_size[1]*self.slice_step_timestamps/self.duration_timestamps))
        mel_specs = []
        for i in range(self.num_segments_sample):
            mel_specs.append(mel_spec_full[0, :, i*slice_step: (i*slice_step) + self.img_size[1]])
        mel_specs = torch.stack(mel_specs, 0)

        if self.separate_norm:
            mel_specs = self.feature_extractor.norm(mel_specs)
        return mel_specs

    def __getitem__(self, ind):

        filename = self.all_filenames[ind]
        wave = self.all_waves[filename]
        
        if self.full_signal:
            mel_specs = self.prepare_mel_specs_from_full_wave(wave=wave)
        else:
            mel_specs = self.prepare_mel_specs_from_segments(wave=wave)
            
        return mel_specs, filename
        

class VINOEngine:
    
    def __init__(self, model, example_dims=(12, 3, 256, 256), ov_model_path=None):
        self.ov_model = self.torch2openvinio(model, example_dims=example_dims, ov_model_path=ov_model_path)
        self.output_layer = -1

    def torch2openvinio(self, model, example_dims, device_name="AUTO", ov_model_path=None):
        core = ov.Core()
        example = torch.randn(example_dims) 
        if ov_model_path is not None:
            ov_model_pytorch = core.read_model(model=ov_model_path)
        else:
            ov_model_pytorch = ov.convert_model(model, example_input=(example,))
        compiled_model_pytorch = core.compile_model(ov_model_pytorch, device_name=device_name)
        return compiled_model_pytorch

    def __call__(self, input):
        result = self.ov_model([input])[self.output_layer]
        return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Configs

In [5]:
def get_multilabel_to_train_labels(multilabel_label2ind, train_label2ind):
    multilabel_to_train_labels = {ind: train_label2ind[label] for label, ind in multilabel_label2ind.items() if label in train_label2ind}
    return multilabel_to_train_labels
    
class InferenceConfig:
    label2ind = {'grekis': 0, 'compau': 1, 'trokin': 2, 'roahaw': 3, 'banana': 4, 'whtdov': 5, 'socfly1': 6, 'yeofly1': 7, 'bobfly1': 8, 'wbwwre1': 9, 'soulap1': 10, 'sobtyr1': 11, 'trsowl': 12, 'laufal1': 13, 'strcuc1': 14, 'bbwduc': 15, 'saffin': 16, 'amekes': 17, 'tropar': 18, 'compot1': 19, 'blbgra1': 20, 'bubwre1': 21, 'strfly1': 22, 'gycwor1': 23, 'greegr': 24, 'linwoo1': 25, 'pirfly1': 26, 'littin1': 27, 'bkmtou1': 28, 'yercac1': 29, 'butsal1': 30, 'smbani': 31, 'bugtan': 32, 'chbant1': 33, 'yebela1': 34, 'rutjac1': 35, 'cotfly1': 36, 'whbman1': 37, 'yehcar1': 38, 'solsan': 39, 'rumfly1': 40, 'yecspi2': 41, 'blhpar1': 42, 'creoro1': 43, 'paltan1': 44, 'rinkin1': 45, 'orcpar': 46, 'stbwoo2': 47, 'speowl1': 48, 'yebfly1': 49, 'plbwoo1': 50, 'yebsee1': 51, 'bkcdon': 52, 'strher': 53, 'y00678': 54, 'babwar': 55, 'strowl1': 56, 'gybmar': 57, 'cocwoo1': 58, 'secfly1': 59, 'thbeup1': 60, 'pavpig2': 61, 'baymac': 62, 'rtlhum': 63, 'purgal2': 64, 'colcha1': 65, 'crcwoo1': 66, 'ywcpar': 67, 'chfmac1': 68, 'rugdov': 69, 'gohman1': 70, 'watjac1': 71, 'grnkin': 72, 'greani1': 73, 'whfant1': 74, 'cattyr': 75, 'srwswa1': 76, 'blbwre1': 77, 'mastit1': 78, 'greibi1': 79, 'snoegr': 80, '41663': 81, 'leagre': 82, 'blcjay1': 83, 'grbhaw1': 84, 'eardov1': 85, 'blcant4': 86, 'whbant1': 87, 'yectyr1': 88, 'rufmot1': 89, 'thlsch3': 90, 'cargra1': 91, 'bicwre1': 92, 'anhing': 93, 'neocor': 94, 'shtfly1': 95, 'recwoo1': 96, 'amakin1': 97, 'ragmac1': 98, 'grasal4': 99, 'gretin1': 100, '65448': 101, 'spepar1': 102, 'fotfly': 103, 'ruther1': 104, 'yehbla2': 105, 'cregua1': 106, '21211': 107, 'whttro1': 108, 'brtpar1': 109, 'rubsee1': 110, 'blkvul': 111, 'verfly': 112, 'cinbec1': 113, 'labter1': 114, 'grepot1': 115, 'palhor2': 116, 'yelori1': 117, '517119': 118, 'colara1': 119, 'crbtan1': 120, 'rebbla1': 121, 'piepuf1': 122, 'savhaw1': 123, 'blchaw1': 124, '22973': 125, 'crebob1': 126, 'whwswa1': 127, 'spbwoo1': 128, '22333': 129, 'bucmot3': 130, '22976': 131, 'tbsfin1': 132, 'cocher1': 133, 'royfly1': 134, 'bobher1': 135, 'olipic1': 136, 'plukit1': 137, 'whmtyr1': 138, 'rosspo1': 139, '52884': 140, '65373': 141, 'blctit1': 142, '50186': 143, 'ampkin1': 144, 'bafibi1': 145, 'woosto': 146, '555086': 147, 'grysee1': 148, '566513': 149, '65962': 150, '48124': 151, 'bubcur1': 152, '42007': 153, 'piwtyr1': 154, 'rutpuf1': 155, '715170': 156, '65349': 157, '65344': 158, '41970': 159, 'shghum1': 160, 'norscr1': 161, 'sahpar1': 162, '67252': 163, '24322': 164, 'turvul': 165, '135045': 166, '65547': 167, '787625': 168, '1462737': 169, 'plctan1': 170, '555142': 171, '126247': 172, '65336': 173, '1564122': 174, '24272': 175, '548639': 176, '46010': 177, '1346504': 178, '963335': 179, '476538': 180, '714022': 181, '66893': 182, '134933': 183, '1192948': 184, '868458': 185, '523060': 186, '24292': 187, '65419': 188, '1194042': 189, '1462711': 190, '81930': 191, '67082': 192, '66578': 193, '66531': 194, '66016': 195, '21038': 196, '41778': 197, '21116': 198, '64862': 199, '528041': 200, '476537': 201, '47067': 202, '42113': 203, '42087': 204, '1139490': 205}
    class_ind2label_inds = {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 124, 126, 127, 128, 130, 132, 133, 134, 135, 136, 137, 138, 139, 142, 144, 145, 146, 148, 152, 154, 155, 160, 161, 162, 165, 170], 1: [101, 107, 118, 125, 129, 131, 141, 147, 150, 157, 158, 163, 164, 166, 167, 168, 171, 172, 173, 175, 180, 182, 183, 187, 188, 191, 192, 193, 194, 195, 196, 198, 199, 201], 2: [81, 149, 153, 159, 177, 197, 202, 203, 204], 3: [140, 143, 151, 156, 169, 174, 176, 178, 179, 181, 184, 185, 186, 189, 190, 200, 205]}
    sample_rate = 32000
    num_classes = len(label2ind)
    num_segments_sample = 12

    # preprocessing
    separate_norm = False
    
    # postprocessing
    smoothing = True
    preds_weights = np.array([0.133, 0.166, 0.133, 0.133, 0.166, 0.133, 0.133])
    preds_power = 1


class ModelsGroupConfig:

    full_signal_to_spectr = True
    duration = 20
    slice_step_sec = 5

    img_size = (224, 512)
    n_mels = img_size[0]
    hop_length = duration * InferenceConfig.sample_rate // (img_size[1] - 1)
    resize = False
    n_fft = 2048*2
    f_min_max = (0, 16000)
    
    example_dims = (InferenceConfig.num_segments_sample, 3, img_size[-2], img_size[-1])
    spectrogram_params = {
                        "n_fft": n_fft,
                        "hop_length": hop_length,
                        "win_length": n_fft,
                        "sample_rate": InferenceConfig.sample_rate,
                        "n_mels": n_mels,
                        "f_min": f_min_max[0],
                        "f_max": f_min_max[1],
                        "normalized": True,
                        "top_db": 80,
                        "sample_mel_normalize": "default",
                        "output_size": None if not resize else img_size
            }

    inference_type = "overlap_average_max_delta"
    

    model_config_1 = {
            "model_type": "sed",
            "example_dims" : example_dims,
            "spectrogram": spectrogram_params,
            "backbone": {
                "backbone_name": 'timm/tf_efficientnet_b3.ns_jft_in1k',
                'pretrained': False,
                "last_layer_hidden_dim": None
            },
            "head": {
                'dropout': 0.50,
                'num_classes': InferenceConfig.num_classes,
                "infer_duration": 5,
                "duration": duration
            },
            "inference_type": inference_type
    }

    model_config_2 = {
            "model_type": "sed",
            "example_dims" : example_dims,
            "spectrogram": spectrogram_params,
            "backbone": {
                "backbone_name": 'timm/regnety_016.tv2_in1k',
                'pretrained': False,
                "last_layer_hidden_dim": None
            },
            "head": {
                'dropout': 0.50,
                'num_classes': InferenceConfig.num_classes,
                "infer_duration": 5,
                "duration": duration
            },
            "inference_type": inference_type
    }

    model_config_3 = {
            "model_type": "sed",
            "example_dims": example_dims,
            "spectrogram": spectrogram_params,
            "backbone": {
                "backbone_name": 'timm/regnety_008.pycls_in1k',
                'pretrained': False,
                "last_layer_hidden_dim": None
            },
            "head": {
                'dropout': 0.50,
                'num_classes': InferenceConfig.num_classes,
                "infer_duration": 5,
                "duration": duration
            },
            "inference_type": inference_type
    }

    label2ind_I_A = {'metrioptera_saussuriana': 0, 'bicolorana_bicolor': 1, 'ephippiger_diurnus': 2, 'eumodicogryllus_bordigalensis': 3, 'decticus_verrucivorus': 4, 'decticus_albifrons': 5, 'roeseliana_roeselii': 6, 'barbitistes_fischeri': 7, 'metaplastes_ornatus': 8, 'platycleis_affinis': 9, 'poecilimon_sanctipauli': 10, 'pseudochorthippus_parallelus': 11, 'chorthippus_biguttulus': 12, 'chorthippus_brunneus': 13, 'chorthippus_mollis': 14, 'eupholidoptera_schmidti': 15, 'gryllus_campestris': 16, 'tessellana_lagrecai': 17, 'saga_hellenica': 18, 'metrioptera_brachyptera': 19, 'oecanthus_pellucens': 20, 'platycleis_albopunctata': 21, 'phaneroptera_falcata': 22, 'canariola_emarginata': 23, 'pholidoptera_griseoaptera': 24, 'platycleis_intermedia': 25, 'pteronemobius_heydenii': 26, 'gryllus_bimaculatus': 27, 'phaneroptera_sparsa': 28, 'yersinella_raymondii': 29, 'tettigonia_viridissima': 30, 'roeseliana_azami': 31, 'tettigonia_cantans': 32, 'poecilimon_jonicus': 33, 'ruspolia_nitidula': 34, 'paragraecia_temasek': 35, 'isophya_lemnotica': 36, 'ancistrura_nigrovittata': 37, 'barbitistes_ocskayi': 38, 'leptophyes_punctatissima': 39, 'tessellana_tessellata': 40, 'pholidoptera_femorata': 41, 'conocephalus_fuscus': 42, 'poecilimon_affinis': 43, 'phaneroptera_nana': 44, 'poecilimon_ornatus': 45, 'tylopsis_lilifolia': 46, 'antaxius_spinibrachius': 47, 'cyrtaspis_scutata': 48, 'uromenus_rugosicollis': 49, 'sepiana_sepium': 50, 'teleogryllus_mitratus': 51, 'nemobius_sylvestris': 52, 'poecilimon_artedentatus': 53, 'neophisis_montealegrei': 54, 'chorthippus_vagans': 55, 'rhacocleis_germanica': 56, '65448': 57, 'chorthippus_jacobsi': 58, 'eupholidoptera_latens': 59, 'gryllotalpa_gryllotalpa': 60, 'pterolepis_spoliata': 61, 'barbitistes_serricauda': 62, '21211': 63, 'platycleis_grisea': 64, 'antaxius_pedestris': 65, 'eugryllodes_pipiens': 66, 'platycleis_sabulosa': 67, 'viriacca_modesta': 68, 'poecilimon_mytilenensis': 69, 'poecilimon_zwicki': 70, 'barbitistes_yersini': 71, 'poecilimon_veluchianus': 72, 'isophya_rhodopensis': 73, 'thyreonotus_corsicus': 74, 'stenobothrus_stigmaticus': 75, 'lluciapomaresius_stalii': 76, 'chorthippus_saulcyi': 77, 'poecilimon_werneri': 78, 'pholidoptera_littoralis': 79, '517119': 80, 'melanogryllus_desertus': 81, 'leptophyes_laticauda': 82, 'sonus_naturalis': 83, 'poecilimon_nobilis': 84, 'stauroderus_scalaris': 85, 'svercus_palmetorum': 86, 'poecilimon_zimmeri': 87, '22973': 88, 'platycleis_falx': 89, 'zeuneriana_abbreviata': 90, 'poecilimon_hamatus': 91, 'ornebius_pullus': 92, 'pelophylax_ridibundus': 93, 'isophya_modestior': 94, 'polysarcus_denticauda': 95, 'chorthippus_apricarius': 96, 'asiophlugis_temasek': 97, 'sorapagus_catalaunicus': 98, 'antaxius_hispanicus': 99, 'pholidoptera_aptera': 100, 'oecanthus_dulcisonans': 101, 'omocestus_viridulus': 102, 'loxoblemmus_parabolicus': 103, '22333': 104, 'stenobothrus_lineatus': 105, 'gryllotalpa_africana': 106, 'leptophyes_albovittata': 107, 'parasteropleurus_perezii': 108, 'andreiniimon_nuptialis': 109, 'poecilimon_hoelzeli': 110, 'barbitistes_obtusus': 111, '22976': 112, 'pholidoptera_fallax': 113, 'acheta_domesticus': 114, 'gryllodes_sigillatus': 115, 'omocestus_raymondi': 116, 'conocephalus_dorsalis': 117, 'isophya_kraussii': 118, 'leptophyes_boscii': 119, 'poecilimon_chopardi': 120, 'casigneta_sp.2': 121, 'asiophlugis_longiuncus': 122, 'varitrella_bakeri': 123, 'chrysochraon_dispar': 124, 'asiophlugis_thaumasia': 125, 'poecilimon_tessellatus': 126, 'loxoblemmus_arietulus': 127, 'gampsocleis_glabra': 128, 'uromenus_brevicollis': 129, 'axylus_philippinus': 130, 'metaplastes_pulchripennis': 131, 'antaxius_difformis': 132, 'isophya_obtusa': 133, 'hyla_arborea': 134, 'alloteratura_lamella': 135, 'borneopsis_cryptosticta': 136, 'eupholidoptera_megastyla': 137, 'platycleis_concii': 138, 'rhacocleis_corsicana': 139, 'tettigonia_caudata': 140, 'poecilimon_ebneri': 141, 'rhacocleis_annulata': 142, 'phaneroptera_brevis': 143, 'isophya_rectipennis': 144, 'natula_averni': 145, 'chorthippus_binotatus': 146, 'peracca_macritchiensis': 147, 'pholidoptera_stankoi': 148, 'ephippiger_ephippiger': 149, 'poecilimon_amissus': 150, 'gryllotalpa_vineae': 151, 'antaxius_kraussii': 152, 'holochlora_bilobata': 153, 'hyla_meridionalis': 154, 'conocephalus_kisi': 155, 'gomphocerippus_rufus': 156, 'parasteropleurus_martorellii': 157, 'omocestus_antigai': 158, 'omocestus_bolivari': 159, 'omocestus_rufipes': 160, 'euchorthippus_declivus': 161, 'platycleis_escalerai': 162, 'salomona_borneensis': 163, 'neocallicrania_selligera': 164, 'chorthippus_dorsatus': 165, 'poecilimon_macedonicus': 166, 'eupholidoptera_smyrnensis': 167, 'stenobothrus_nigromaculatus': 168, 'poecilimon_fussii': 169, 'stethophyma_grossum': 170, 'incertana_incerta': 171, 'zvenella_geniculata': 172, 'poecilimon_laevissimus': 173, 'conocephalus_conocephalus': 174, 'pycnogaster_jugicola': 175, 'isophya_pyrenaea': 176, '52884': 177, 'isophya_tosevski': 178, 'ectadia_fulva': 179, 'ephippiger_terrestris': 180, 'poecilimon_orbelicus': 181, 'parasongella_ornaticeps': 182, '65373': 183, 'albarracinia_zapaterii': 184, 'cycloptiloides_timah': 185, 'myrmeleotettix_maculatus': 186, 'baetica_ustulata': 187, 'oecanthus_indicus': 188, 'steropleurus_andalusius': 189, '50186': 190, 'epidalea_calamita': 191, 'velarifictorus_aspersus': 192, 'vichetia_oblongicollis': 193, 'velarifictorus_tenepalpus': 194, 'yersinella_beybienkoi': 195, 'xenogryllus_transversus': 196, 'thyreonotus_bidens': 197, 'isophya_hospodar': 198, 'bufotes_viridis': 199, 'pelophylax_lessonae': 200, 'ornebius_tampines': 201, 'poecilimon_propinquus': 202, 'chorthippus_eisentrauti': 203, 'asiophlugis_trusmadi': 204, 'omocestus_haemorrhoidalis': 205, 'trellius_helverseni': 206, 'metrioptera_buyssoni': 207, 'poecilimon_superbus': 208, 'steropleurus_flavovittatus': 209, 'amedegnatiana_vicheti': 210, 'poecilimon_schmidtii': 211, 'odontura_stenoxypha': 212, 'polionemobius_taprobanense': 213, 'arcyptera_fusca': 214, 'bufo_bufo': 215, 'callicrania_ramburii': 216, 'leptophyes_lisae': 217, 'acrometopa_macropoda': 218, 'poecilimon_sureyanus': 219, 'poecilimon_gracilis': 220, 'pterolepis_cordubensis': 221, 'lluciapomaresius_asturiensis': 222, 'rhacocleis_distinguenda': 223, 'oecanthus_allardi': 224, 'poecilimon_obesus': 225, 'phyllomimus_inversus': 226, 'lucasinova_nigromarginata': 227, 'modestana_ebneri': 228, 'conocephalus_semivittatus': 229, 'ornebius_aureus': 230, 'poecilimon_thoracicus': 231, 'acrometopa_italica': 232, 'poecilimon_thessalicus': 233, 'ephippigerida_areolaria': 234, 'poecilimon_elegans': 235, 'ducetia_japonica': 236, 'odontura_macphersoni': 237, 'synephippius_obvius': 238, 'stenobothrus_rubicundulus': 239, 'eupholidoptera_chabrieri': 240, 'ornebius_insculptus': 241, '555086': 242, 'bombina_bombina': 243, 'chorthippus_corsicus': 244, 'gymnogryllus_angustus': 245, 'tettigonia_hispanica': 246, 'ctenodecticus_thymi': 247, 'gomphocerus_sibiricus': 248, 'odontura_aspericauda': 249, 'bradyporus_dasypus': 250, 'metaplastes_oertzeni': 251, 'pholidoptera_macedonica': 252, 'teleogryllus_oceanicus': 253, 'pachytrachis_gracilis': 254, 'poecilimon_mariannae': 255, 'ornebius_rufonigrus': 256, 'salomona_maculifrons': 257, 'isophya_plevnensis': 258, 'roeseliana_oporina': 259, 'parnassiana_tymphrestos': 260, 'platycleis_iberica': 261, 'antaxius_chopardi': 262, 'corsteropleurus_chopardi': 263, 'pteronemobius_lineolatus': 264, 'ectatoderus_argentatus': 265, 'noctitrella_plurilingua': 266, 'tettigonia_longispina': 267, "pelophylax_'esculentus'": 268, 'gryllus_assimilis': 269, 'stenobothrus_festivus': 270, 'alytes_obstetricans': 271, '65962': 272, 'pseudacris_crucifer': 273, 'euthystira_brachyptera': 274, 'rana_arvalis': 275, 'poecilimon_marmaraensis': 276, 'homoeoxipha_lycoides': 277, 'rhacocleis_bonfilsi': 278, 'rhacocleis_edentata': 279, 'anonconotus_ghilianii': 280, '48124': 281, 'leptophyes_sicula': 282, 'arcyptera_tornosi': 283, 'scudderia_furcata': 284, 'anonconotus_ligustinus': 285, 'eupholidoptera_tyrrhenica': 286, 'chorthippus_albomarginatus': 287, 'odontura_glabricauda': 288, 'gampsocleis_abbreviata': 289, 'neocallicrania_miegii': 290, 'eugryllodes_escalerae': 291, 'poecilimon_paros': 292, 'anomaloglossus_baeobatrachus': 293, 'anonconotus_mercantouri': 294, 'modicogryllus_algirius': 295, 'isophya_longicaudata': 296, 'anonconotus_alpinus': 297, 'poecilimon_turcicus': 298, 'uromenus_elegans': 299, 'eumodicogryllus_theryi': 300, 'viriacca_insularis': 301, 'isophya_modesta': 302, 'conocephalus_albescens': 303, 'holochlora_nigrotympana': 304, 'broughtonia_domogledi': 305, 'tartarogryllus_tartarus': 306, 'zeuneriana_burriana': 307, 'tessellana_orina': 308, 'ephippiger_ruffoi': 309, 'sciobia_caliendra': 310, 'pelophylax_perezi': 311, 'lipotactes_maculatus': 312, 'poecilimon_luschani': 313, 'pelodytes_punctatus': 314, '715170': 315, 'euanisous_teuthroides': 316, 'poecilimon_antalyaensis': 317, 'rana_temporaria': 318, 'helicocercus_triguttatus': 319, 'ornebius_bioculatus': 320, 'ectatoderus_angusticollis': 321, 'psophus_stridulus': 322, 'psyrana_tigrina': 323, 'anonconotus_baracunensis': 324, 'kuzicus_denticulatus': 325, 'saga_natoliae': 326, 'eupholidoptera_garganica': 327, 'lluciapomaresius_ortegai': 328, 'stenobothrus_fischeri': 329, 'sabaterpia_taeniata': 330, 'casigneta_sp.1': 331, 'poecilimon_pergamicus': 332, 'conocephalus_cognatus': 333, 'hexacentrus_unicolor': 334, '65344': 335, '65349': 336, 'eupholidoptera_cypria': 337, 'lipotactes_kabili': 338, 'eupholidoptera_giuliae': 339, 'arnobia_ocellata': 340, 'teleogryllus_occipitalis': 341, 'aulacobothrus_taeniatus': 342, 'tarbinskiellus_portentosus': 343, 'rhacocleis_poneli': 344, 'antaxius_sorrezensis': 345, 'pelophylax_kurtmuelleri': 346, 'gampsocleis_sedakovii': 347, 'bicolorana_kraussi': 348, 'stictophaula_armata': 349, 'acrometopa_servillea': 350, 'chorthippus_dubius': 351, 'mecopoda_elongata': 352, 'platystolus_martinezii': 353, 'parnassiana_chelmos': 354, 'montana_stricta': 355, 'poecilimon_ampliatus': 356, 'polysarcus_scutatus': 357, 'asiophlugis_philippina': 358, 'saga_campbelli': 359, 'eupholidoptera_mariannae': 360, 'antaxius_florezi': 361, 'tettigonia_silana': 362, 'chorthippus_yersini': 363, 'mitius_blennus': 364, 'oecanthus_nigricornis': 365, 'hylarana_leptoglossa': 366, 'paratrigonidium_venustulum': 367, '67252': 368, 'euchorthippus_elegantulus': 369, 'amphiestris_baetica': 370, 'euchorthippus_chopardi': 371, 'oedipoda_caerulescens': 372, 'ephippiger_melisi': 373, '24322': 374, 'uromenus_agarenus': 375, 'poecilimon_pindos': 376, 'lithobates_palustris': 377, 'isophya_savignyi': 378, 'anadrymadusa_brevipennis': 379, 'neoconocephalus_triops': 380, 'chorthippus_bornhalmi': 381, 'isophya_pavelii': 382, 'ephippiger_persicarius': 383, 'epacromius_pulverulentus': 384, 'pterolepis_grallata': 385, 'ctenodecticus_ramburi': 386, 'lipotactes_alienus': 387, 'poecilimon_erimanthos': 388, 'modicogryllus_frontalis': 389, 'conocephalus_exemptus': 390, 'parnassiana_tenuis': 391, 'isophya_mavromoustakisi': 392, 'pantecphylus_cerambycinus': 393, 'calliphona_koenigi': 394, 'metrioptera_ambigua': 395, 'ramburiella_hispanica': 396, 'acrometopa_syriaca': 397, 'incertana_decorata': 398, 'ctenodecticus_granatensis': 399, 'isophya_costata': 400, 'parnassiana_parnassica': 401, 'dociostaurus_maroccanus': 402, 'neophisis_siamensis': 403, 'bufo_spinosus': 404, 'isophya_speciosa': 405, 'hyla_sarda': 406, 'leptophyes_discoidalis': 407, 'rana_dalmatina': 408, 'zvenella_transversa': 409, 'drymadusa_dorsalis': 410, 'pachytrachis_striolatus': 411, 'conocephalus_cinereus': 412, 'pycnogaster_inermis': 413, 'conocephalus_concolor': 414, 'isophya_major': 415, 'arcyptera_microptera': 416, 'callicrania_plaxicauda': 417, 'bucephaloptera_bucephala': 418, 'bryodemella_holdereri': 419, 'bradyporus_oniscus': 420, 'eupholidoptera_forcipata': 421, 'pseudacris_feriarum': 422, 'angaracris_barabensis': 423, 'anadrymadusa_ornatipennis': 424, 'ctenodecticus_major': 425, 'isophya_pienensis': 426, 'velarifictorus_micado': 427, 'pholidoptera_frivaldszkyi': 428, 'parnassiana_dirphys': 429, 'oecanthus_quadripunctatus': 430, 'varitrella_glabra': 431, 'orchelimum_nigripes': 432, 'chiasmocleis_haddadi': 433, 'gampsocleis_gratiosa': 434, 'truljalia_versicolor': 435, 'liara_magna': 436, 'turanogryllus_lateralis': 437, 'otophryne_pyburni': 438, 'xiphidiopsis_dicera': 439, 'zvenella_yunnana': 440, 'metrioptera_maritima': 441, 'tympanophyllum_arcufolium': 442, 'rhammatocerus_cyanipes': 443, 'chorthippus_nevadensis': 444, 'fritziana_goeldii': 445, '135045': 446, 'rhacocleis_graeca': 447, 'poecilimon_unispinosus': 448, 'barbitistes_constrictus': 449, 'poecilimon_lodosi': 450, 'stenobothrus_grammicus': 451, 'poecilimon_brunneri': 452, 'poecilimon_miramae': 453, 'chorthippus_apicalis': 454, 'rhacocleis_baccettii': 455, 'eupholidoptera_astyla': 456, 'dociostaurus_jagoi': 457, 'psorodonotus_illyricus': 458, 'parnassiana_tymphiensis': 459, 'bombina_variegata': 460, 'acrometopa_cretensis': 461, 'lipotactes_virescens': 462, 'velarifictorus_acutilobus': 463, 'parnassiana_fusca': 464, 'scambophyllum_sanguinolentum': 465, 'pristimantis_espedeus': 466, 'hyla_molleri': 467, 'arcyptera_kheili': 468, 'hyla_orientalis': 469, 'poecilimon_roseoviridis': 470, 'steropleurus_brunnerii': 471, 'omocestus_uhagonii': 472, 'incertana_drepanensis': 473, 'poecilimon_martinae': 474, 'antaxius_bouvieri': 475, 'neophisis_longipennis': 476, 'dryophytes_versicolor': 477, 'omocestus_panteli': 478, 'ameerega_hahneli': 479, 'chorthippus_rubratibialis': 480, 'metrioptera_caprai': 481, 'ephippiger_provincialis': 482, 'promeca_sumatrana': 483, 'chorthippus_reissingeri': 484, 'chorthippus_macrocerus': 485, 'chorthippus_maritimus': 486, 'phonochorion_uvarovi': 487, 'gryllodinus_kerkennensis': 488, 'rhacocleis_thyrrhenica': 489, 'natula_longipennis': 490, 'psorodonotus_macedonicus': 491, 'psorodonotus_fieberi': 492, 'zeuneriana_amplipennis': 493, 'isophya_camptoxypha': 494, 'isophya_brunneri': 495, 'chorthippus_jucundus': 496, 'micrornebius_inopinatus': 497, 'parnassiana_gionica': 498, 'osteocephalus_oophagus': 499, 'rhacocleis_werneri': 500, 'rhacocleis_lithoscirtetes': 501, 'isophya_rizeensis': 502, 'metrioptera_prenjica': 503, 'anelytra_curvata': 504, 'loxoblemmus_jacobsoni': 505, 'anterastes_serbicus': 506, 'macroxiphus_sumatranus': 507, 'micrornebius_maninjau': 508, 'hyla_intermedia': 509, 'ornebius_samudra': 510, 'decticus_aprutianus': 511, 'acheta_hispanicus': 512, 'pholidoptera_lucasi': 513, 'dryophytes_cinereus': 514, 'omocestus_petraeus': 515, 'aquarana_clamitans': 516, 'hyla_savignyi': 517, 'parnassiana_parnon': 518, 'eugaster_guyoni': 519, 'rhacocleis_neglecta': 520, 'glyphonotus_coniciplicus': 521, 'rhinella_marina': 522, 'adenomera_heyeri': 523, 'anaxyrus_americanus': 524, 'stenobothrus_bolivarii': 525, 'poecilimon_heroicus': 526, 'pelodytes_ibericus': 527, 'prionotropis_appula': 528, 'aeropedellus_variegatus': 529, '65547': 530, 'arcyptera_brevipennis': 531, 'pycnogaster_sanchezgomezi': 532, 'trachycephalus_hadroceps': 533, 'eupholidoptera_magnifica': 534, 'eugaster_spinulosa': 535, '1462737': 536, 'anonconotus_occidentalis': 537, 'duolandrevus_firmus': 538, 'pholidoptera_dalmatica': 539, 'ephippiger_discoidalis': 540, '787625': 541, 'pholidoptera_transsylvanica': 542, 'leptodactylus_mystaceus': 543, '65336': 544, 'isophya_posthumoidalis': 545, 'sardoplatycleis_galvagnii': 546, 'rhinella_ornata': 547, 'anelytra_fastigata': 548, 'anelytra_tristellata': 549, 'barbitistes_kaltenbachi': 550, 'bradyporus_avanos': 551, 'montana_tomini': 552, 'eleutherodactylus_planirostris': 553, 'bradyporus_macrogaster': 554, 'omocestus_minutissimus': 555, 'tettigonia_armeniaca': 556, 'xenogryllus_ululiu': 557, 'meconematini_sp.': 558, 'velarifictorus_brevifrons': 559, 'pseudacris_regilla': 560, 'isophya_salmani': 561, 'phyllomimus_truncatus': 562, '1564122': 563, 'microhyla_ornata': 564, '24272': 565, 'poecilimon_celebi': 566, 'decticus_loudoni': 567, 'italohippus_monticola': 568, 'pseudacris_collinsorum': 569, 'rhacocleis_derrai': 570, 'rhacocleis_crypta': 571, 'aquarana_grylio': 572, 'chorthippus_dichrous': 573, 'pseudacris_maculata': 574, '555142': 575, 'loxoblemmus_doenitzi': 576, 'phaulula_malayica': 577, 'pholidoptera_rhodopensis': 578, 'phaneroptera_spinosa': 579, 'parnassiana_coracis': 580, 'velarifictorus_horridus': 581, 'isophya_straubei': 582, 'truxalis_siamensis': 583, 'atelopus_flavescens': 584, 'chorthippus_acroleucus': 585, 'tessellana_nigrosignata': 586, 'montana_barretii': 587, '126247': 588, 'parnassiana_menalon': 589, 'loxoblemmus_equestris': 590, 'leurophyllum_brevicauda': 591, 'acris_gryllus': 592, '548639': 593, 'smilisca_baudinii': 594, '66893': 595, '714022': 596, 'hylarana_guentheri': 597, 'odontura_maroccana': 598, '1346504': 599, 'pithecopus_rohdei': 600, '476538': 601, 'arcyptera_albogeniculata': 602, 'rhaebo_guttatus': 603, 'montana_macedonica': 604, 'pseudochorthippus_montanus': 605, 'uromenus_maroccanus': 606, 'hyla_chinensis': 607, 'promeca_perakana': 608, 'montana_armeniaca': 609, 'microcentrum_angustatum': 610, 'chorthippus_pullus': 611, 'sphingonotus_corsicus': 612, 'rhacocleis_insularis': 613, 'pterophylla_camellifolia': 614, 'psorodonotus_specularis': 615, 'oecanthus_longicauda': 616, 'adenomera_andreae': 617, 'oecanthus_euryelytra': 618, 'poecilimon_pseudornatus': 619, 'uromenus_siculus': 620, 'omocestus_africanus': 621, 'platycleis_romana': 622, 'pycnogaster_gaditana': 623, 'anaxipha_exigua': 624, 'meconema_meridionale': 625, 'lithoxenus_heptapotamica': 626, 'isophya_yaraligozi': 627, 'poecilimon_isopterus': 628, '963335': 629, 'pelophylax_nigromaculatus': 630, 'omocestus_minutus': 631, 'deracantha_onos': 632, 'aquarana_catesbeiana': 633, 'myrmeleotettix_palpalis': 634, 'ololygon_perpusilla': 635, 'hyperolius_spinigularis': 636, 'fritziana_mitus': 637, 'scinax_nebulosus': 638, 'leptodactylus_stenodema': 639, '134933': 640, '523060': 641, '1192948': 642, 'microhyla_heymonsi': 643, 'pseudacris_ocularis': 644, 'fejervarya_multistriata': 645, 'dryophytes_chrysoscelis': 646, 'velarifictorus_sulcifrons': 647, 'litoria_fallax': 648, 'boana_punctata': 649, 'dendropsophus_minusculus': 650, '868458': 651, 'pelobates_fuscus': 652, 'euphlyctis_cyanophlyctis': 653, '65419': 654, 'lithobates_sphenocephalus': 655, 'dryophytes_squirellus': 656, 'hyperolius_concolor': 657, 'phasmahyla_guttata': 658, 'brachycephalus_hermogenesi': 659, '24292': 660, 'leptodactylus_wagneri': 661, 'scinax_boesemani': 662, 'boana_cinerascens': 663, '1462711': 664, '1194042': 665, 'pelophylax_bedriagae': 666, 'dendropsophus_elegans': 667, 'scinax_imbegue': 668, 'alytes_almogavarii': 669, 'gastrophryne_carolinensis': 670, '66531': 671, '66016': 672, '21116': 673, '66578': 674, '67082': 675, 'dendropsophus_gaucheri': 676, '81930': 677, 'boana_raniceps': 678, 'boana_albomarginata': 679, 'trachycephalus_resinifictrix': 680, 'duttaphrynus_melanostictus': 681, 'boana_faber': 682, '21038': 683, '1139490': 684, '528041': 685, 'ischnocnema_bolbodactyla': 686, '64862': 687, 'sphaenorhynchus_lacteus': 688, '476537': 689, 'epipedobates_anthonyi': 690, 'rhinella_granulosa': 691, 'hylodes_phyllodes': 692, 'alytes_dickhilleni': 693, 'boana_multifasciata': 694, 'ololygon_littoralis': 695, 'scinax_hayii': 696, 'alytes_cisternasii': 697, 'pristimantis_zeuctotylus': 698, 'dendropsophus_minutus': 699}
    model_config_4 = {
            "model_type": "sed",
            "example_dims": example_dims,
            "spectrogram": spectrogram_params,
            "backbone": {
                "backbone_name": 'tf_efficientnet_b0.ns_jft_in1k',
                'pretrained': False,
                "last_layer_hidden_dim": None
            },
            "head": {
                'dropout': 0.50,
                'num_classes': len(label2ind_I_A),
                "infer_duration": 5,
                "duration": duration
            },
            "inference_type": inference_type,
            "multilabel_to_train_labels": get_multilabel_to_train_labels(multilabel_label2ind=label2ind_I_A, train_label2ind=InferenceConfig.label2ind)
    }

    model_config_5 = {
            "model_type": "sed",
            "example_dims": example_dims,
            "spectrogram": spectrogram_params,
            "backbone": {
                "backbone_name": 'timm/eca_nfnet_l0.ra2_in1k',
                'pretrained': False,
                "last_layer_hidden_dim": None
            },
            "head": {
                'dropout': 0.50,
                'num_classes': InferenceConfig.num_classes,
                "infer_duration": 5,
                "duration": duration
            },
            "inference_type": inference_type
    }

    model_config_6 = {
            "model_type": "sed",
            "example_dims": example_dims,
            "spectrogram": spectrogram_params,
            "backbone": {
                "backbone_name": 'tf_efficientnet_b4.ns_jft_in1k',
                'pretrained': False,
                "last_layer_hidden_dim": None
            },
            "head": {
                'dropout': 0.50,
                'num_classes': InferenceConfig.num_classes,
                "infer_duration": 5,
                "duration": duration
            },
            "inference_type": inference_type
    }

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data Loading

In [6]:
DATA_PATH = "/kaggle/input/birdclef-2025"
TEST_DIR =  "test_soundscapes"
TRAIN_DIR =  "train_soundscapes"

test_paths = glob(os.path.join(DATA_PATH, TEST_DIR, '*ogg'))
if len(test_paths) == 0:
    test_paths = glob(os.path.join(DATA_PATH, TRAIN_DIR, '*ogg'))[:3]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df['filename'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.ogg',''))
test_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,filepath,filename
0,/kaggle/input/birdclef-2025/train_soundscapes/...,H27_20230421_155000
1,/kaggle/input/birdclef-2025/train_soundscapes/...,H09_20230424_014500
2,/kaggle/input/birdclef-2025/train_soundscapes/...,H78_20230512_071000


In [7]:
def load_all_samples(test_df):
    
    def load_sample(filepath, sample_rate=32_000, res_type="kaiser_best"):
        wave, _ = librosa.load(filepath, sr=sample_rate, res_type=res_type)
        return wave
    
    waves = joblib.Parallel(n_jobs=os.cpu_count())(
            joblib.delayed(load_sample)(
                filepath,
            )
            for filepath in tqdm.notebook.tqdm(test_df['filepath'].values)
        )
    all_waves = dict(zip(test_df['filename'].values, waves))
    
    return all_waves
    
all_waves = load_all_samples(test_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/3 [00:00<?, ?it/s]

# Models building

In [8]:
def prepare_models(models_group, models_dirs, ov_models_dir=None):
    models = []
    ov_engines = []
    for weights_name, model_config  in tqdm.notebook.tqdm(models_group):
        
        model = CLEFClassifierSED(config=model_config)
        pt_path = os.path.join(models_dirs, weights_name)
        model.load_state_dict(torch.load(pt_path, weights_only=True, map_location=torch.device('cpu')))
        model.eval()

        if ov_models_dir is not None:
            backbone_ov_path = os.path.join(ov_models_dir, weights_name.replace(".pt", ".xml"))
            if not os.path.exists(backbone_ov_path):
                backbone_ov_path = None
        else:
            backbone_ov_path = None
            
        ov_engine = VINOEngine(model.backbone, 
                               example_dims=model_config["example_dims"],
                               ov_model_path=backbone_ov_path)
        models.append(model)
        ov_engines.append(ov_engine)

    models = {
        "models": models,
        "ov_engines": ov_engines
    }
    return models
    

MODELS_PATH = "/kaggle/input/birdclef2025-1st-place-ensemble"
OV_MODELS_PATH = "/kaggle/input/birdclef2025-1st-place-ensemble"

MODELS_GROUP_META_1 = [
    
    # self-learning models
    ("tf_efficientnet_b4.ns_jft_in1k_sampler_maxsum_iteration_3_v1_temp_0.55_64_bs_0.15_drop_path_rate_1_mixup_ratio_pseudo_data_20_duration_sed_type_0.5_mixup_p_(224, 512)_size_ce_4096_n_fft_0_fold_22_seed_25_epoch.pt", ModelsGroupConfig.model_config_6),
    ("tf_efficientnet_b3.ns_jft_in1k_sampler_maxsum_iteration_3_v1_temp_0.55_54_bs_0.15_drop_path_rate_1_mixup_ratio_pseudo_data_20_duration_sed_type_0.5_mixup_p_(224, 512)_size_ce_4096_n_fft_1_fold_25_epoch.pt", ModelsGroupConfig.model_config_1),
    ("regnety_016.tv2_in1k_sampler_maxsum_iteration_4_v1_temp_0.6_64_bs_0.15_drop_path_rate_1_mixup_ratio_pseudo_data_20_duration_sed_type_0.5_mixup_p_(224, 512)_size_ce_4096_n_fft_2_fold_25_epoch.pt", ModelsGroupConfig.model_config_2),
    ("regnety_016.tv2_in1k_sampler_maxsum_iteration_4_v1_framewise_temp_0.6_64_bs_0.15_drop_path_rate_1_mixup_ratio_pseudo_data_20_duration_sed_type_0.5_mixup_p_(224, 512)_size_ce_4096_n_fft_3_fold_25_epoch.pt", ModelsGroupConfig.model_config_2),
    ("eca_nfnet_l0.ra2_in1k_sampler_maxsum_iteration_3_v1_temp_0.55_128_bs_0.15_drop_path_rate_1_mixup_ratio_pseudo_data_20_duration_sed_type_0.5_mixup_p_(224, 512)_size_ce_4096_n_fft_additional_data_full_data_22_seed_15_epoch.pt", ModelsGroupConfig.model_config_5),
    
    # supervised learning models
    ("regnety_008.pycls_in1k_20_duration_sed_mixup_(224, 512)_size_ce_4096_n_fft_2_fold_15_epoch.pt", ModelsGroupConfig.model_config_3),
    ("tf_efficientnet_b0.ns_jft_in1k_incest_amphibia_128_bs_0.0_drop_path_rate_20_duration_sed_type_0.5_mixup_p_(224, 512)_size_ce_4096_n_fft_full_data_22_seed_40_epoch.pt", ModelsGroupConfig.model_config_4)
]
models_group_1 = prepare_models(models_group=MODELS_GROUP_META_1, models_dirs=MODELS_PATH, ov_models_dir=OV_MODELS_PATH)
models_group_1["dataset"] =  InferenceDataset(all_waves=all_waves, 
                                            feature_extractor=models_group_1["models"][0].mel_spectr_generator, 
                                            
                                            full_signal=ModelsGroupConfig.full_signal_to_spectr,
                                            hop_length=ModelsGroupConfig.hop_length, 
                                            img_size=ModelsGroupConfig.img_size,
                                            duration_sec=ModelsGroupConfig.duration, 
                                            slice_step_sec=ModelsGroupConfig.slice_step_sec,
                                            
                                            num_segments_sample=InferenceConfig.num_segments_sample, 
                                            separate_norm=InferenceConfig.separate_norm)

model_groups = [models_group_1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/7 [00:00<?, ?it/s]

# Inference

In [9]:
def gauss_convolve(arr):
    weights = np.array([0.1, 0.2, 0.4, 0.2, 0.1])
    result = convolve1d(arr, weights, axis=0, mode='nearest')
    return result


def ensemble_preds(preds, weights, power=1):
    preds = preds**power
    preds = preds * weights.reshape(-1, 1, 1)
    preds = preds.sum(0)
    return preds
    

def multilabel_pred_to_train_preds(preds, multilabel_to_train_labels):
    if isinstance(multilabel_to_train_labels, dict):
        y = np.zeros((InferenceConfig.num_segments_sample, len(InferenceConfig.label2ind)), dtype=np.float32)
        for multilabel_ind, train_ind in multilabel_to_train_labels.items():
            y[:, train_ind] = preds[:, multilabel_ind]
        return y
    else:
        return preds
    

def inference_model(mel_spec, model, ov_engine):
    fts = ov_engine(mel_spec)
    fts = torch.from_numpy(fts)
    with torch.no_grad():
        with torch.autocast(device_type="cpu", dtype=torch.float16):
            preds = model.get_head_preds(fts) 
    preds = preds.astype(np.float32)
    preds = multilabel_pred_to_train_preds(preds, multilabel_to_train_labels=model.multilabel_to_train_labels)
    return preds


def inference_sample(sample_ind, model_groups):

    sample_probs = []
    for model_group in model_groups:
        mel_spec, filename = model_group['dataset'][sample_ind]
        if len(mel_spec.shape) == 3:
            mel_spec = mel_spec.unsqueeze(1).expand(-1,3,-1,-1)
        for model, ov_engine in zip(model_group["models"], model_group["ov_engines"]):
            probs = inference_model(mel_spec, model=model, ov_engine=ov_engine)
            sample_probs.append(probs)

    sample_probs = ensemble_preds(preds=np.array(sample_probs),
                                  weights=InferenceConfig.preds_weights,
                                  power=InferenceConfig.preds_power)
    if InferenceConfig.smoothing:
        sample_probs = gauss_convolve(sample_probs)
    
    rec_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(InferenceConfig.num_segments_sample)]

    return sample_probs, rec_ids
    

preds_all = []
ids = []
for sample_ind in tqdm.tqdm(range(len(all_waves))):
    sample_probs, rec_ids = inference_sample(sample_ind=sample_ind, model_groups=model_groups)
    preds_all.append(sample_probs)
    ids.extend(rec_ids)

if len(preds_all) > 0:
    preds_all = np.concatenate(preds_all, 0)
    pred_df = pd.DataFrame(ids, columns=['row_id'])
    pred_df.loc[:, list(InferenceConfig.label2ind.keys())] = preds_all
else:
    pred_df = pd.DataFrame(columns=['row_id']+list(InferenceConfig.label2ind.keys()))
    
pred_df.to_csv('submission.csv',index=False)
pred_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 3/3 [00:30<00:00, 10.24s/it]


,row_id,grekis,compau,trokin,roahaw,banana,whtdov,socfly1,yeofly1,bobfly1,...,21038,41778,21116,64862,528041,476537,47067,42113,42087,1139490
0,H27_20230421_155000_5,0.122161,0.005220,0.060943,0.152163,0.009912,0.028203,0.031689,0.008058,0.279805,...,0.000005,0.000366,0.000071,0.000303,0.000054,0.000146,0.000054,0.000070,0.000009,0.000083
1,H27_20230421_155000_10,0.101886,0.007072,0.084604,0.107305,0.010258,0.028640,0.039335,0.013196,0.158775,...,0.000009,0.000585,0.000073,0.000910,0.000086,0.000286,0.000080,0.000088,0.000009,0.000119
2,H27_20230421_155000_15,0.096395,0.010187,0.099079,0.103817,0.009468,0.034481,0.048094,0.015870,0.085098,...,0.000014,0.000887,0.000093,0.001591,0.000126,0.000361,0.000108,0.000109,0.000012,0.000175
3,H27_20230421_155000_20,0.113508,0.011246,0.094181,0.173667,0.008337,0.031275,0.052119,0.013576,0.060784,...,0.000028,0.000824,0.000116,0.002262,0.000159,0.000310,0.000107,0.000118,0.000011,0.000245
4,H27_20230421_155000_25,0.150842,0.011664,0.067578,0.269226,0.007060,0.026250,0.041632,0.010011,0.066650,...,0.000044,0.000759,0.000128,0.001624,0.000130,0.000190,0.000096,0.000077,0.000009,0.000213
